In [1]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.211:7077") \
        .appName("Marina_Mota_Project")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 1)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [2]:
df = spark_session.read.load("hdfs://localhost:9000/user/ubuntu/input/RC_2005-12.bz2", format="json")
#file = spark_context.textFile("hdfs://localhost:9000/user/ubuntu/input/RC_2005-12.bz2")

In [3]:
df.show()

+----------+----------------------+-----------------+--------------------+----------------+-----------+-------------+------+------+---+--------+---------+------------+-----+--------+----------+------------+---+
|    author|author_flair_css_class|author_flair_text|                body|controversiality|created_utc|distinguished|edited|gilded| id| link_id|parent_id|retrieved_on|score|stickied| subreddit|subreddit_id|ups|
+----------+----------------------+-----------------+--------------------+----------------+-----------+-------------+------+------+---+--------+---------+------------+-----+--------+----------+------------+---+
|      frjo|                  null|             null|A look at Vietnam...|               0| 1134365188|         null| false|     0|c13|t3_17863| t3_17863|  1473738411|    2|   false|reddit.com|        t5_6|  2|
|   zse7zse|                  null|             null|The site states "...|               0| 1134365725|         null| false|     0|c14|t3_17866| t3_17866|  

In [4]:
# release the cores for another application!
spark_context.stop()